# Baseline Models for Credit Card Fraud Detection

Kaggle project:

https://www.kaggle.com/mlg-ulb/creditcardfraud

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
ls data

creditcard.csv


In [3]:
df = pd.read_csv('data/creditcard.csv')
df.shape

(284807, 31)

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
# Very imbalanced classification

df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [7]:
# Check null values

df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [12]:
# split the data, stratify based on df.Class

from sklearn.model_selection import train_test_split


train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df.Class)

In [14]:
train.Class.value_counts()

0    227451
1       394
Name: Class, dtype: int64

In [15]:
test.Class.value_counts()

0    56864
1       98
Name: Class, dtype: int64

In [24]:
X_train = train.iloc[:,:-1]
y_train = train.Class

X_test = test.iloc[:,:-1]
y_test = test.Class

In [28]:
from sklearn import linear_model
from sklearn.metrics import classification_report

model = linear_model.LogisticRegressionCV()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.82      0.64      0.72        98

    accuracy                           1.00     56962
   macro avg       0.91      0.82      0.86     56962
weighted avg       1.00      1.00      1.00     56962



In [31]:
# Balance the data, pretty bad

model = linear_model.LogisticRegressionCV(class_weight = "balanced")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.91      0.12        98

    accuracy                           0.98     56962
   macro avg       0.53      0.94      0.55     56962
weighted avg       1.00      0.98      0.99     56962



In [34]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler(feature_range = (-1, 1))

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
# rescale the features, looks a little more accurate

model = linear_model.LogisticRegressionCV()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.65      0.73        98

    accuracy                           1.00     56962
   macro avg       0.92      0.83      0.87     56962
weighted avg       1.00      1.00      1.00     56962



In [40]:
# XGBoost looks good

import xgboost

model = xgboost.XGBClassifier()  
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

[13:57:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.93      0.82      0.87        98

    accuracy                           1.00     56962
   macro avg       0.96      0.91      0.93     56962
weighted avg       1.00      1.00      1.00     56962



In [42]:
# lightgbm pretty bad

import lightgbm

model = lightgbm.LGBMClassifier()  
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56864
           1       0.17      0.64      0.26        98

    accuracy                           0.99     56962
   macro avg       0.58      0.82      0.63     56962
weighted avg       1.00      0.99      1.00     56962



In [43]:
from sklearn import ensemble

model = ensemble.RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.94      0.82      0.87        98

    accuracy                           1.00     56962
   macro avg       0.97      0.91      0.94     56962
weighted avg       1.00      1.00      1.00     56962



In [45]:
#from sklearn.model_selection import GridSearchCV

#param_grid = { 
    #'bootstrap': [True],
    #'n_estimators': [100, 200, 300],
    #'max_features': ['auto', 'sqrt', 'log2'],
    #'max_depth': [3, 5, 10],
    #'criterion' :['gini', 'entropy']
#}

# Create a based model
#rf = ensemble.RandomForestClassifier()
# Instantiate the grid search model
#clf  = GridSearchCV(rf, cv=5, param_grid = param_grid)
#clf.fit(X_train, y_train)